In [1]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [2]:
#del sys.modules["utilities"]
from utilities import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [5]:
df_feats_surgeryprocess = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_SurgeryProcess.pkl','rb'))
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_feats_icd_transform = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_icd_transform.pkl','rb'))
df_preor_ICULOS = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_preor_ICULOS.pkl','rb'))
df_feats_delirium = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_delirium.pkl','rb'))

df_icu_graph_features = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_icu_graph_features.pkl','rb'))

df_feats_ppg_train_all = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_train_separate.pkl','rb'))
df_feats_ppg_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_val_separate.pkl','rb'))
df_feats_ppg_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_test.pkl','rb'))


feats_n2v_all_train = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_train.pkl','rb'))
feats_n2v_all_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_val.pkl','rb'))
feats_n2v_all_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_test.pkl','rb'))



Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [6]:
df_feats_generalclinical_process_icug = df_feats_clinical.merge(df_feats_icd_transform,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_preor_ICULOS,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_feats_delirium,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_feats_surgeryprocess,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_icu_graph_features,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left')

In [7]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [8]:
df_feats_generalclinical_process_ppg_icug_train = df_feats_generalclinical_process_icug[df_feats_generalclinical_process_icug['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_train_all,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_train,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_generalclinical_process_ppg_icug_val = df_feats_generalclinical_process_icug[df_feats_generalclinical_process_icug['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_val,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_generalclinical_process_ppg_icug_test = df_feats_generalclinical_process_icug[df_feats_generalclinical_process_icug['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_test,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


len(df_feats_generalclinical_process_ppg_icug_train)+len(df_feats_generalclinical_process_ppg_icug_val)+len(df_feats_generalclinical_process_ppg_icug_test)-len(df_feats_generalclinical_process_icug)

0

In [9]:
y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_test = Outcome_labels_test

X_train = y_train[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_generalclinical_process_ppg_icug_train,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)
X_val = y_val[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_generalclinical_process_ppg_icug_val,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_test = y_test[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_generalclinical_process_ppg_icug_test,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_train_val = pd.concat([X_train,X_val]).reset_index(drop=True)
y_train_val = pd.concat([y_train,y_val]).reset_index(drop=True)

In [10]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 4520),
 (13665, 4520),
 (122983, 4520),
 (13664, 4520),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [11]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [12]:
del sys.modules["utilities"]
from utilities import *

In [13]:
#experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[2023])

In [14]:
np.random.seed(666)

In [15]:
a = np.zeros(len(X_train))-1 
b = np.ones(len(X_val)) 
pds = PredefinedSplit(np.concatenate([a,b]))

In [16]:
pipe = Pipeline(steps=[
                       ("imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
                       ("scaler", MinMaxScaler()),
                       ("classifier", MLPClassifier()) #default 3-layer mlp
                        ])

In [17]:
clf = BayesSearchCV(estimator = pipe,
                   search_spaces={
                    'classifier__alpha':Real(0.0001,0.99,'log-uniform'),
                    'classifier__learning_rate': Categorical(['constant','adaptive']),
                    'classifier__early_stopping': Categorical([True,False])
                   },
                   n_iter = 30,
                   cv=pds,
                   scoring = 'roc_auc',
                   n_jobs=32,
                   refit=True,
                   verbose=4,
                   random_state=666
                   ) 

In [18]:
clf.fit(X_train_val.iloc[:,1:],y_train_val.iloc[:,1:].values.ravel())

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fi

BayesSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
              estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                        ('scaler', MinMaxScaler()),
                                        ('classifier', MLPClassifier())]),
              n_iter=30, n_jobs=32, random_state=666, scoring='roc_auc',
              search_spaces={'classifier__alpha': Real(low=0.0001, high=0.99, prior='log-uniform', transform='normalize'),
                             'classifier__early_stopping': Categorical(categories=(True, False), prior=None),
                             'classifier__learning_rate': Categorical(categories=('constant', 'adaptive'), prior=None)},
              verbose=4)

In [19]:
print(clf.best_params_)
best_m = clf.best_estimator_
pred = best_m.predict(X_test.iloc[:,1:])
pred_prob = best_m.predict_proba(X_test.iloc[:,1:])[:,1]
print(min(pred_prob),max(pred_prob))

print('Best Validation AUROC: {}'.format(clf.best_score_))

print('Test AUROC: {}'.format(roc_auc_score(y_test.iloc[:,1:], pred_prob)))
print('Test F1-score: {}'.format(f1_score(y_test.iloc[:,1:], pred)))
print('Test Recall: {}'.format(recall_score(y_test.iloc[:,1:], pred)))
print('Test Precision: {}'.format(precision_score(y_test.iloc[:,1:], pred)))
print('Test Accuracy: {}'.format(accuracy_score(y_test.iloc[:,1:], pred)))

OrderedDict([('classifier__alpha', 0.1009782709585233), ('classifier__early_stopping', False), ('classifier__learning_rate', 'adaptive')])
6.870655730216285e-06 0.9100085818809769
Best Validation AUROC: 0.8722199623352166
Test AUROC: 0.854735066653311
Test F1-score: 0.6344770337576092
Test Recall: 0.5813894523326572
Test Precision: 0.6982338611449452
Test Accuracy: 0.8066451990632318
